# Clustering Neighborhoods in Toronto

Assignment to explore, segment, and cluster the neighborhoods in the city of Toronto. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Toronto Dataset</a>

2. <a href="#item2">Venues in Downtown Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood in Downtown Toronto</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

## Libraries

In [44]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Toronto Dataset

Toronto dataset will consist of 5 columns: PostalCode, Borough, Neighborhood, Latitude and Longitude.

* Dataset taken from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
* BeautifulSoup package used to parse the data. 
* Code to retrieve data is here: *******

In [45]:
import pandas as pd

In [46]:
df_toronto_nb = pd.read_csv('toronto_neighborhoods.csv')
df_toronto_nb.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [47]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto_nb['Borough'].unique()), df_toronto_nb.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


### Geographical coordinates of  Toronto

In [48]:
# To avoid error GeocoderServiceError: [SSL: CERTIFICATE_VERIFY_FAILED]
import certifi
import ssl
import geopy.geocoders 
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

In [49]:
def get_latitude_longitude(address, agent): 
    geolocator = Nominatim(user_agent=agent)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

In [50]:
latitude, longitude = get_latitude_longitude('Toronto, ON', "on_explorer")
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude)) 

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Toronto map with neighborhoods|

In [51]:
def create_neighborhood_map(latitude, longitude, df_data, zoom=10, color='green'):
    map_view = folium.Map(location=[latitude, longitude], zoom_start=zoom)

    # add markers to map
    for lat, lng, borough, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Borough'], df_data['Neighborhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_view)  
        
    return map_view

In [52]:
map_toronto = create_neighborhood_map(latitude, longitude, df_toronto_nb)
map_toronto

## Downtown Toronto

In [53]:
down_toronto = df_toronto_nb[df_toronto_nb['Borough']=='Downtown Toronto']
down_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [54]:
print('Downtown Toronto has {} neighborhoods.'.format(down_toronto.shape[0]))

Downtown Toronto has 19 neighborhoods.


In [55]:
latitude_dt, longitude_dt = get_latitude_longitude('Downtown Toronto, ON', "on_explorer")
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude_dt, longitude_dt)) 

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [56]:
map_down_toronto = create_neighborhood_map(latitude_dt, longitude_dt, down_toronto, zoom=12, color='limegreen')
map_down_toronto


## Foursquare API

<!-- Credentials:
CLIENT_ID = '0TPBBOGOU5SJALY0M2DW3E4WBO5TXJCMDROYNFDQVYPVQLST'  
CLIENT_SECRET = 'AYQ0CHZTPKTKTDCTFNFPJIMWM4DPHJFDAB3XJJWOP45MF05Q' 
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100 
--> 

code copied from NY lab-week3:

In [57]:
CLIENT_ID = '0TPBBOGOU5SJALY0M2DW3E4WBO5TXJCMDROYNFDQVYPVQLST' # your Foursquare ID
CLIENT_SECRET = 'AYQ0CHZTPKTKTDCTFNFPJIMWM4DPHJFDAB3XJJWOP45MF05Q' # your Foursquare Secret
VERSION = '20200424'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0TPBBOGOU5SJALY0M2DW3E4WBO5TXJCMDROYNFDQVYPVQLST
CLIENT_SECRET:AYQ0CHZTPKTKTDCTFNFPJIMWM4DPHJFDAB3XJJWOP45MF05Q


In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<a id='item2'></a>

## 2. Venues in Downtown Toronto

Getting venues using Foursquare API

In [59]:
down_toronto_venues = getNearbyVenues(names=down_toronto['Neighborhood'],
                                   latitudes=down_toronto['Latitude'],
                                   longitudes=down_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [60]:
print('Downtown Toronto has {} venues.'.format(len(down_toronto_venues['Venue'].unique())))

Downtown Toronto has 379 venues.


In [61]:
down_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood

In [62]:
down_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
"First Canadian Place, Underground city",30,30,30,30,30,30
"Garden District, Ryerson",30,30,30,30,30,30
"Harbourfront East, Union Station, Toronto Islands",30,30,30,30,30,30


Let's find out how many unique categories can be curated from all the returned venues

In [63]:
print('Downtown Toronto has {} uniques categories in {} neighborhoods'.format(
    len(down_toronto_venues['Venue Category'].unique()),
    len(down_toronto_venues['Neighborhood'].unique())))

Downtown Toronto has 154 uniques categories in 19 neighborhoods


<a id='item3'></a>

## 3. Analyze Each Neighborhood in Downtown Toronto

In [68]:
# one hot encoding
toronto_onehot = pd.get_dummies(down_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

if 'Neighborhood' in toronto_onehot.columns.tolist():
    toronto_onehot.drop(columns=['Neighborhood'], inplace=True)
    
# add neighborhood column back to dataframe
toronto_onehot.insert(0, 'Neighborhood', down_toronto_venues['Neighborhood'])
toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(518, 154)

In [69]:
toronto_onehot.describe()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
count,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000
mean,0.001931,0.001931,0.001931,0.003861,0.003861,0.001931,0.009653,0.001931,0.015444,0.001931,0.001931,0.003861,0.001931,0.001931,0.001931,0.028958,0.005792,0.01

### Mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.033333,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.033333,0.0,0.000000,0.033333,0.066667,0.0,0.0,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.066667,0.100000,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.033333,0.0,0.033333,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.066667,0.0,0.033333,0.0,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.033333,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033333,0.0,0.0,0.0000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,0.0,0.0,0.000000,0.000000,0.000000,0.066667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.000000,0.0,0.033333,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.117647,0.058824,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.000000,0.0,0.058824,0.000000,0.000000,0.0,0.0,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.058824,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.058824,0.058824,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.058824,0.0,0.0,0.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top 7 venues for each neighborhood.

In [27]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Museum
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport,Historic Site,Plane,Rental Car Location,Boutique
2,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Park,Sandwich Place,Ramen Restaurant,Bubble Tea Shop
3,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Candy Store,Italian Restaurant
4,Church and Wellesley,Coffee Shop,Gym,Burger Joint,Creperie,Bookstore,Salon / Barbershop,Breakfast Spot


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 4, 0, 3, 3, 3, 3, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = down_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Performing Arts Venue,Chocolate Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Yoga Studio,Diner,Portuguese Restaurant,Park,Mexican Restaurant,Italian Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Café,Clothing Store,Theater,Pizza Place,Bookstore,Ramen Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Gastropub,Coffee Shop,Café,Restaurant,Japanese Restaurant,Farmers Market,BBQ Joint
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Farmers Market,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Museum


Finally, let's visualize the resulting clusters

In [30]:
# create map
map_clusters = folium.Map(location=[latitude_dt, longitude_dt], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [31]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Performing Arts Venue,Chocolate Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Yoga Studio,Diner,Portuguese Restaurant,Park,Mexican Restaurant,Italian Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Café,Clothing Store,Theater,Pizza Place,Bookstore,Ramen Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Gastropub,Coffee Shop,Café,Restaurant,Japanese Restaurant,Farmers Market,BBQ Joint
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Farmers Market,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Museum


In [32]:
# Filter: Neighborhood and Most-common columns
select_cols = list(filter(lambda x: 'Neighborhood' in x or 'Most' in x, toronto_merged.columns))

In [33]:
def get_cluster_info(df, cluster, select_cols):
    df_cl = df[df['Cluster Labels'] == cluster ][select_cols]
    n_venues = df_cl.shape[0]
    venue_info = '' if n_venues <= 1 else '\n* The 3 Most common venues are:\n\n{}'.format(df_cl.stack().value_counts()[:3])
    print_info = ('Cluster {}:\n\n* Has {} neighborhoods.{}'.format(cluster+1,n_venues,venue_info))
    return df_cl, print_info

### Exploring Clusters:

#### Cluster 1: red

In [34]:
cl1_df, cl1_info = get_cluster_info(toronto_merged, 0, select_cols)
print(cl1_info)

Cluster 1:

* Has 5 neighborhoods.
* The 3 Most common venues are:

Beer Bar       3
Coffee Shop    2
Creperie       2
dtype: int64


In [35]:
cl1_df.style.hide_index()

Matplotlib is building the font cache; this may take a moment.


Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
Berczy Park,Coffee Shop,Farmers Market,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Museum
"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Mexican Restaurant,Belgian Restaurant,Jazz Club,Beer Bar,Dumpling Restaurant
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Airport Lounge,Airport Service,Airport,Historic Site,Plane,Rental Car Location,Boutique
Stn A PO Boxes,Beer Bar,Creperie,Cocktail Bar,Farmers Market,Bakery,Park,Museum
Church and Wellesley,Coffee Shop,Gym,Burger Joint,Creperie,Bookstore,Salon / Barbershop,Breakfast Spot




#### Cluster 2: purple

In [36]:
cl2_df, cl2_info = get_cluster_info(toronto_merged, 1, select_cols)
print(cl2_info)

Cluster 2:

* Has 1 neighborhoods.


In [37]:
cl2_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
91,Rosedale,Park,Playground,Trail,Tennis Court,Cocktail Bar,Comfort Food Restaurant,Deli / Bodega




#### Cluster 3: blue

In [38]:
cl3_df, cl3_info = get_cluster_info(toronto_merged, 2, select_cols)
print(cl3_info)

Cluster 3:

* Has 3 neighborhoods.
* The 3 Most common venues are:

Park                  3
Coffee Shop           3
Italian Restaurant    2
dtype: int64


In [39]:
cl3_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Performing Arts Venue,Chocolate Shop
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,Diner,Portuguese Restaurant,Park,Mexican Restaurant,Italian Restaurant
24,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Park,Sandwich Place,Ramen Restaurant,Bubble Tea Shop



#### Cluster 4: lightgreen

In [40]:
cl4_df, cl4_info = get_cluster_info(toronto_merged, 3, select_cols)
print(cl4_info)

Cluster 4:

* Has 9 neighborhoods.
* The 3 Most common venues are:

Café           9
Coffee Shop    7
Restaurant     5
dtype: int64


In [41]:
cl4_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
9,"Garden District, Ryerson",Coffee Shop,Café,Clothing Store,Theater,Pizza Place,Bookstore,Ramen Restaurant
15,St. James Town,Gastropub,Coffee Shop,Café,Restaurant,Japanese Restaurant,Farmers Market,BBQ Joint
30,"Richmond, Adelaide, King",Café,Steakhouse,Coffee Shop,Seafood Restaurant,Pizza Place,Colombian Restaurant,Lounge
36,"Harbourfront East, Union Station, Toronto Islands",Park,Plaza,Café,Hotel,Japanese Restaurant,Lake,Skating Rink
42,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Restaurant,Café,Japanese Restaurant,Pizza Place,Bar,Beer Bar
48,"Commerce Court, Victoria Hotel",Café,Hotel,Gastropub,Restaurant,Coffee Shop,Japanese Restaurant,Gym
80,"University of Toronto, Harbord",Café,Bookstore,Sandwich Place,Bar,Bakery,Japanese Restaurant,Yoga Studio
96,"St. James Town, Cabbagetown",Restaurant,Bakery,Italian Restaurant,Café,Coffee Shop,Market,Butcher
97,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Seafood Restaurant,Gym,Steakhouse,Gluten-free Restaurant


#### Cluster 5: orange

In [42]:
cl5_df, cl5_info = get_cluster_info(toronto_merged, 4, select_cols)
print(cl5_info)

Cluster 5:

* Has 1 neighborhoods.


In [43]:
cl5_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
25,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Candy Store,Italian Restaurant



### Acknowledgment

This notebook was based on the NY cluseting notebook created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/).

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).